In [1]:
import gurobipy as gp

In [6]:
def get_num_districts(state):

    return None

In [16]:
state = 'RI'
filename = "districting_2020\\2020\\{}\\counties\\graph\\{}.population".format(state, state)


with open('Numberofdistricts.txt', 'r') as file:
    for line in file:
        parts = line.strip().split('\t')
        if parts[0] == state:
            num_districts =  int(parts[1])

with open(filename, "r") as file:
    # Skip the first line
    next(file)
    # Count the number of lines, each line represents a county
    num_counties = sum(1 for line in file)



2

In [19]:
# Model
m = gp.Model("district")

state = 'RI'
adjacency_matrix_file = r'districting_2020\2020\RI\counties\graph\RI_distances.csv'

# Set decision variable x for each node j wether it is in district k
for i in range(num_counties):
    for j in range(num_districts):
        x = m.addVar(vtype=gp.GRB.BINARY, name="x_{}_{}".format(i, j))

# Set decision variable y for each pair of nodes i and k, and each district j
for i in range(num_counties):
    for k in range(num_counties):
        for j in range(num_districts):
            y = m.addVar(vtype=gp.GRB.BINARY, name="y_{}_{}_{}".format(i, k, j))





